In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip uninstall -qqy jupyterlab kfp  # Remove unused conflicting packages
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 75.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [4]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [5]:
DOCUMENT1 = "AI in Medical Imaging and Disease Diagnosis  Artificial Intelligence (AI) is transforming medical imaging and disease diagnosis by providing faster, more accurate analysis of medical scans. Traditional diagnostic methods often rely on human interpretation, which can be time-consuming and subject to human error. AI-powered medical imaging solutions use deep learning algorithms trained on vast datasets of X-rays, MRIs, and CT scans to identify abnormalities such as tumors, fractures, and infections with high accuracy.  One significant application of AI in medical imaging is the early detection of cancers, such as lung and breast cancer. AI models have been trained to detect subtle patterns in mammograms and CT scans that might be missed by radiologists. In one study, AI outperformed human radiologists in detecting breast cancer, reducing false positives and minimizing unnecessary biopsies.  AI also plays a critical role in neuroimaging, assisting in the diagnosis of conditions like Alzheimer's disease, stroke, and multiple sclerosis. Deep learning models analyze MRI scans to detect early signs of cognitive decline, enabling early interventions and better patient outcomes.  Another breakthrough in AI-driven medical imaging is its use in ophthalmology. AI-powered retinal imaging systems can detect diabetic retinopathy, a leading cause of blindness, by analyzing retinal scans for signs of damage. This allows for early treatment and reduces the risk of vision loss.  Overall, AI in medical imaging enhances diagnostic accuracy, speeds up workflow, and provides valuable insights for doctors. With continuous advancements in deep learning and computer vision, AI-driven imaging tools are expected to become an integral part of clinical practice."

DOCUMENT2 = "AI in Predictive Analytics and Personalized Medicine  Predictive analytics in healthcare uses AI to analyze vast amounts of patient data and identify patterns that indicate potential health risks. AI-driven predictive models process electronic health records (EHRs), genetic information, and lifestyle data to forecast the likelihood of diseases such as diabetes, heart disease, and Alzheimer’s. This allows for early interventions, personalized treatment plans, and improved patient outcomes.  One major area where AI is making a difference is in cardiology. AI models trained on electrocardiograms (ECGs) and echocardiograms can detect early signs of heart disease, including arrhythmias and heart failure, before symptoms appear. By analyzing a patient’s medical history and risk factors, AI can predict the probability of a heart attack, allowing doctors to take preventive measures.  In oncology, AI is revolutionizing cancer treatment by enabling personalized medicine. AI algorithms analyze a patient’s genetic profile to determine the most effective cancer treatment, reducing trial-and-error approaches and improving survival rates. For example, AI-driven precision oncology tools match cancer patients with targeted therapies based on their unique genetic mutations.  AI also plays a role in predicting disease outbreaks and pandemics. By analyzing global health data, AI models can identify patterns in the spread of infectious diseases, enabling governments and healthcare organizations to implement timely interventions. For instance, AI was used to track and predict the spread of COVID-19 by analyzing data from online reports, travel patterns, and hospital records.  Overall, AI-driven predictive analytics enhances early disease detection, enables precision medicine, and optimizes healthcare delivery. With continuous improvements in AI models, healthcare providers can make more informed decisions, ultimately leading to better patient care and outcomes."

DOCUMENT3 = "AI in Drug Discovery and Clinical Trials  The process of discovering new drugs is traditionally slow, costly, and requires years of research and testing. AI is transforming drug discovery by analyzing massive datasets, predicting potential drug candidates, and optimizing clinical trials, significantly accelerating the process.  AI-driven drug discovery involves machine learning models that analyze chemical structures and predict how they interact with biological systems. By screening millions of compounds in silico, AI helps researchers identify promising drug candidates much faster than traditional methods. For example, AI was instrumental in identifying potential treatments for COVID-19 by analyzing existing drugs that could be repurposed.  AI is also revolutionizing clinical trials by optimizing patient recruitment and improving trial efficiency. One of the major challenges in clinical trials is finding suitable participants who meet specific criteria. AI-driven tools analyze electronic health records and genetic data to identify the best candidates, ensuring trials are conducted with the most relevant patient groups. This reduces the time and cost associated with traditional recruitment methods.  Additionally, AI enhances the monitoring of clinical trials by analyzing real-time patient data. Wearable devices and sensors collect continuous health data, which AI processes to identify potential adverse effects or treatment responses. This enables researchers to make data-driven adjustments to trials, ensuring patient safety and improving outcomes.  Beyond drug discovery and trials, AI is used in precision medicine to develop personalized treatment plans. By analyzing a patient’s genetic makeup, lifestyle, and medical history, AI can recommend the most effective drugs with minimal side effects. This approach has been particularly beneficial in cancer treatment, where targeted therapies are designed based on a patient’s unique tumor profile.  In summary, AI accelerates drug discovery, optimizes clinical trials, and enables precision medicine. As AI continues to advance, the pharmaceutical industry will benefit from faster, more efficient drug development, ultimately leading to better treatments for patients."


documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [6]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/gemini-embedding-exp-03-07",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

In [7]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [8]:
db.count()
# You can peek at the data too.
# db.peek(1)

3

In [9]:
db.peek(1)

{'ids': ['0'],
 'embeddings': array([[-0.00785584,  0.00185402, -0.00313931, ..., -0.01716148,
         -0.0067208 , -0.00444444]]),
 'documents': ["AI in Medical Imaging and Disease Diagnosis  Artificial Intelligence (AI) is transforming medical imaging and disease diagnosis by providing faster, more accurate analysis of medical scans. Traditional diagnostic methods often rely on human interpretation, which can be time-consuming and subject to human error. AI-powered medical imaging solutions use deep learning algorithms trained on vast datasets of X-rays, MRIs, and CT scans to identify abnormalities such as tumors, fractures, and infections with high accuracy.  One significant application of AI in medical imaging is the early detection of cancers, such as lung and breast cancer. AI models have been trained to detect subtle patterns in mammograms and CT scans that might be missed by radiologists. In one study, AI outperformed human radiologists in detecting breast cancer, reducing fal

In [10]:
#Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "How does AI help in diagnosing diseases?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

AI in Medical Imaging and Disease Diagnosis  Artificial Intelligence (AI) is transforming medical imaging and disease diagnosis by providing faster, more accurate analysis of medical scans. Traditional diagnostic methods often rely on human interpretation, which can be time-consuming and subject to human error. AI-powered medical imaging solutions use deep learning algorithms trained on vast datasets of X-rays, MRIs, and CT scans to identify abnormalities such as tumors, fractures, and infections with high accuracy.  One significant application of AI in medical imaging is the early detection of cancers, such as lung and breast cancer. AI models have been trained to detect subtle patterns in mammograms and CT scans that might be missed by radiologists. In one study, AI outperformed human radiologists in detecting breast cancer, reducing false positives and minimizing unnecessary biopsies.  AI also plays a critical role in neuroimaging, assisting in the diagnosis of conditions like Alzheimer's disease, stroke, and multiple sclerosis. Deep learning models analyze MRI scans to detect early signs of cognitive decline, enabling early interventions and better patient outcomes.  Another breakthrough in AI-driven medical imaging is its use in ophthalmology. AI-powered retinal imaging systems can detect diabetic retinopathy, a leading cause of blindness, by analyzing retinal scans for signs of damage. This allows for early treatment and reduces the risk of vision loss.  Overall, AI in medical imaging enhances diagnostic accuracy, speeds up workflow, and provides valuable insights for doctors. With continuous advancements in deep learning and computer vision, AI-driven imaging tools are expected to become an integral part of clinical practice.

In [11]:
#Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "Can AI help in clinical trials?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

AI in Drug Discovery and Clinical Trials  The process of discovering new drugs is traditionally slow, costly, and requires years of research and testing. AI is transforming drug discovery by analyzing massive datasets, predicting potential drug candidates, and optimizing clinical trials, significantly accelerating the process.  AI-driven drug discovery involves machine learning models that analyze chemical structures and predict how they interact with biological systems. By screening millions of compounds in silico, AI helps researchers identify promising drug candidates much faster than traditional methods. For example, AI was instrumental in identifying potential treatments for COVID-19 by analyzing existing drugs that could be repurposed.  AI is also revolutionizing clinical trials by optimizing patient recruitment and improving trial efficiency. One of the major challenges in clinical trials is finding suitable participants who meet specific criteria. AI-driven tools analyze electronic health records and genetic data to identify the best candidates, ensuring trials are conducted with the most relevant patient groups. This reduces the time and cost associated with traditional recruitment methods.  Additionally, AI enhances the monitoring of clinical trials by analyzing real-time patient data. Wearable devices and sensors collect continuous health data, which AI processes to identify potential adverse effects or treatment responses. This enables researchers to make data-driven adjustments to trials, ensuring patient safety and improving outcomes.  Beyond drug discovery and trials, AI is used in precision medicine to develop personalized treatment plans. By analyzing a patient’s genetic makeup, lifestyle, and medical history, AI can recommend the most effective drugs with minimal side effects. This approach has been particularly beneficial in cancer treatment, where targeted therapies are designed based on a patient’s unique tumor profile.  In summary, AI accelerates drug discovery, optimizes clinical trials, and enables precision medicine. As AI continues to advance, the pharmaceutical industry will benefit from faster, more efficient drug development, ultimately leading to better treatments for patients.

In [12]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative AI bot specialised in healthcare that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience and provide advise based on your understanding of the refernce above . and if you are unsure about some answers look up the informations from external relevant source to give an appropriate answers, so be sure to break down complicated concepts and 
strike a friendly and converstional tone.and also provide some external links to the source materials where the users can learn more about it If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative AI bot specialised in healthcare that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience and provide advise based on your understanding of the refernce above . and if you are unsure about some answers look up the informations from external relevant source to give an appropriate answers, so be sure to break down complicated concepts and 
strike a friendly and converstional tone.and also provide some external links to the source materials where the users can learn more about it If the passage is irrelevant to the answer, you may ignore it.

QUESTION: Can AI help in clinical trials?
PASSAGE: AI in Drug Discovery and Clinical Trials  The process of discovering new drugs is traditionally slow, costly, and requires years of research and testing. AI is transforming drug disco

In [13]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

Yes, AI can indeed be a game-changer in clinical trials, making the whole process faster, more efficient, and safer for patients. Here's how:

*   **Speeding up Drug Discovery:** Traditionally, finding new drugs is a long and expensive journey. AI can sift through huge amounts of data to predict potential drug candidates and fine-tune clinical trials, which really speeds things up.

*   **Finding the Right Participants:** One of the biggest hurdles in clinical trials is finding the right people who fit specific criteria. AI can analyze electronic health records and genetic data to pinpoint the best candidates, saving time and money in the recruitment process.

*   **Keeping a Close Watch on Patients:** AI can continuously monitor patients in real-time using wearable devices and sensors. By analyzing this data, researchers can quickly spot any potential side effects or how well the treatment is working, allowing them to make timely adjustments to the trial for patient safety and better results.

*   **Personalized Treatment Plans:** AI also helps in precision medicine by creating personalized treatment plans. It looks at a patient's genes, lifestyle, and medical history to recommend the most effective drugs with the fewest side effects. This is especially useful in cancer treatment, where therapies can be tailored to a patient's unique tumor profile.

In short, AI is revolutionizing clinical trials by speeding up drug discovery, optimizing patient recruitment, and enabling personalized treatment plans. As AI gets even better, the pharmaceutical industry will benefit from quicker, more efficient drug development, which ultimately means better treatments for patients.


In [15]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.9 MB/s eta 0:00:00


In [19]:
!pip install --upgrade python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.7 MB/s eta 0:00:00


In [35]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 27.2 MB/s eta 0:00:0000:010:01m


In [33]:
!pip install pypdf langchain


  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.


In [49]:
import gradio as gr
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

# 🧠 Your AI Healthcare Chatbot Function
def ai_healthcare_bot(user_query):
    # Search Chroma DB (as you already do)
    result = db.query(query_texts=[user_query], n_results=1)
    [all_passages] = result["documents"]

    # Create Prompt
    query_oneline = user_query.replace("\n", " ")
    prompt = f"""You are a helpful and informative AI bot specialised in healthcare that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience and provide advise based on your understanding of the refernce above. If you are unsure about some answers, look up the information from external relevant sources to give an appropriate answer. 
Be sure to break down complicated concepts and strike a friendly and conversational tone. Also, provide some external links to the source materials where users can learn more about it. 

QUESTION: {query_oneline}
"""

    for passage in all_passages:
        passage_oneline = passage.replace("\n", " ")
        prompt += f"PASSAGE: {passage_oneline}\n"

    # Generate AI response using Gemini
    answer = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )

    return f"### 🤖 AI Response:\n\n{answer.text}"

# 📄 New Function: Summarize Uploaded PDF
def summarize_pdf(file):
    reader = PdfReader(file.name)
    raw_text = ""
    for page in reader.pages:
        raw_text += page.extract_text()

    # Chunk text for better summarization
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1500, chunk_overlap=200)
    chunks = splitter.split_text(raw_text)

    # Create summarization prompt
    prompt = """
    You are a helpful and knowledgeable AI assistant specialized in healthcare. Your task is to read and summarize the information from the uploaded document in a clear, friendly, and easy-to-understand way. 

Ensure your summary is comprehensive, includes key details, and presents important background information when needed. Your audience is non-technical, so break down complex medical or technical terms into simple explanations. Strike a conversational tone — imagine you're explaining it to someone without a healthcare background.

If there are specific topics, medical terms, or research findings mentioned, briefly explain what they mean and why they matter. Whenever possible, include links to reliable external sources or articles for further reading, so users can learn more on their own.

If the document covers multiple topics, organize your summary into clear sections or bullet points for readability.

Avoid guessing or fabricating details — if anything is unclear, just mention that it wasn't explained in detail.

Summarize the following document content:
    """

    for chunk in chunks:
        prompt += f"{chunk}\n"

    # Generate summary using Gemini
    summary = client.models.generate_content(
        model="gemini-2.0-flash",  # You can use pro model here for better results
        contents=prompt
    )

    return f"## 📄 Summary of the PDF:\n\n{summary.text}"

# 🎨 Gradio Interface with Tabs
with gr.Blocks() as interface:
    gr.Markdown("## 🤖 AI in Healthcare Chatbot")

    with gr.Tab("💬 Ask a Healthcare Question"):
        user_input = gr.Textbox(label="Ask me about AI in Healthcare")
        output = gr.Markdown()  # Using Markdown for formatted responses
        ask_button = gr.Button("Submit")
        ask_button.click(ai_healthcare_bot, inputs=user_input, outputs=output)

    with gr.Tab("📄 Upload PDF & Summarize it using AI"):
        file_input = gr.File(label="Upload your PDF here", file_types=[".pdf"])
        summary_output = gr.Markdown()  # Using Markdown for formatted summaries
        summarize_button = gr.Button("Summarize PDF")
        summarize_button.click(summarize_pdf, inputs=file_input, outputs=summary_output)

# 🚀 Launch App
interface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://a83650f9029243e937.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [47]:
!sudo apt update && sudo apt install python3-pip
!pip install gradio google-generativeai langchain chromadb


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]3m
Get:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease   m
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease                        
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]       
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [70.9 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,978 kB]m
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]          
Get:12 https://developer.download.nvidia.com/compute/cuda/rep